In [9]:
!pip install xeno-canto

In [16]:
# Download only the metadata
import json

!xeno-canto -m grp:"bats" q:"A"

# Uncomment the next line to get all data (WARNING 10 GiB)
# !xeno-canto -dl grp:"bats" q:"A"

dir = "dataset/metadata/grp_batsq_A"

with open(dir+"/page1.json", 'r') as file:
    data = json.load(file)
    print("API message:", data["message"])
    print("Number of recordings:", data["numRecordings"])
    print("Number of pages:", data["numPages"])

API message: Xeno-canto API v2 will be phased out on October 10, 2025. Visit https://xeno-canto.org/explore/api for API v3 documentation.
Number of recordings: 1171
Number of pages: 3


In [36]:
import os
import json
import pandas as pd

# Convert metadata into frames
frames = []
for entry in os.scandir(dir):
    if entry.is_file():
        with open(entry, 'r') as file:
            data = json.load(file)
            df = pd.DataFrame(data['recordings'])
            frames.append(df)

all_data = pd.concat(frames)

# Print some metadata
print (all_data.columns.values.tolist())


# all_data.head()

# print(all_data)


['id', 'gen', 'sp', 'ssp', 'group', 'en', 'rec', 'cnt', 'loc', 'lat', 'lng', 'alt', 'type', 'sex', 'stage', 'method', 'url', 'file', 'file-name', 'sono', 'osci', 'lic', 'q', 'length', 'time', 'date', 'uploaded', 'also', 'rmk', 'bird-seen', 'animal-seen', 'playback-used', 'temp', 'regnr', 'auto', 'dvc', 'mic', 'smp']


In [35]:
data_selection = all_data[["id", "gen", "sp", "en", "cnt", "loc", "type", "sex", "length", "dvc", "mic"]]
data_selection.head()

,id,gen,sp,en,cnt,loc,type,sex,length,dvc,mic
0,920292,Eptesicus,serotinus,Eurasian Serotine,Netherlands,"Stramproy, Weert, Limburg","echolocation, feeding buzz",,0:26,Echo Meter Touch 2,
1,914886,Eptesicus,serotinus,Eurasian Serotine,Netherlands,"Klundert, Moerdijk, Noord-Brabant","echolocation, social call",,0:14,Echo Meter Touch 2,
2,913356,Eptesicus,serotinus,Eurasian Serotine,Netherlands,"Roelofarendsveen, Waterryck/Gemeentehuis (near...","echolocation, feeding buzz",,0:14,Echo Meter Touch 2,
3,913125,Eptesicus,serotinus,Eurasian Serotine,Germany,"Havelland (near Rathenow), Brandenburg","echolocation, feeding buzz",,0:29,Gigaset GX4,Echo Meter Touch 2 Pro
4,913114,Eptesicus,serotinus,Eurasian Serotine,Germany,"Havelland (near Rathenow), Brandenburg","echolocation, feeding buzz",,0:15,Gigaset GX4,Echo Meter Touch 2 Pro
